In [25]:
import pandas as pd
import json

In [26]:
def filter_zero_columns(dataframe): #delete columnds that are 0.0 down the entire line
    for column in dataframe:
        if sum(dataframe[column]) == 0.0:
            del dataframe[column] 

In [32]:
benchmark = 'sunflow'
iter = 10
type = 'c-linklist'

filename_base = f'{benchmark}_{iter}_{type}'
with open(f'{filename_base}-metadata.json') as fh: metainfo = json.loads(fh.read())
data = pd.read_csv(f'~/throwawayfiles/AsyncMonitor-C.csv')
#filter_zero_columns(data)

print(metainfo)
data

{'samplingRate': 0, 'lifetime': 20341, 'numSamples': 285126}


,socket,dram,core,pkg,timestamp
0,1,6139.7567,0.0,190462.2191,1615049974135676
1,2,45040.5217,0.0,171821.9373,1615049974135720
2,1,6139.7567,0.0,190463.4612,1615049974262874
3,2,45040.6551,0.0,171823.8879,1615049974262918
4,1,6139.7567,0.0,190464.5654,1615049974390064
5,2,45040.7663,0.0,171825.6902,1615049974390110
6,1,6139.7584,0.0,190465.6581,1615049974517260
7,2,45040.8810,0.0,171827.4981,1615049974517306
8,1,6139.7584,0.0,190466.7944,1615049974644454
9,2,45040.9995,0.0,171829.3207,1615049974644499


In [47]:
for i in range(max(data['socket'])):
    socket = i+1
    print(socket)
    data_currsock = data[data.socket.eq(socket)]
    del data_currsock['socket']
    filter_zero_columns(data_currsock)
    print(data_currsock)
print(data)

1
         dram          pkg         timestamp
0   6139.7567  190462.2191  1615049974135676
2   6139.7567  190463.4612  1615049974262874
4   6139.7567  190464.5654  1615049974390064
6   6139.7584  190465.6581  1615049974517260
8   6139.7584  190466.7944  1615049974644454
10  6139.7584  190467.9473  1615049974771646
12  6139.7584  190469.0717  1615049974898839
14  6139.7584  190470.1533  1615049975026034
16  6139.7592  190471.2219  1615049975153226
18  6139.7592  190472.3759  1615049975280418
20  6139.7592  190473.5186  1615049975407604
22  6139.7592  190474.6646  1615049975534801
24  6139.7592  190475.8087  1615049975661992
26  6139.7592  190476.9948  1615049975789184
28  6139.7592  190478.1660  1615049975916378
30  6139.7592  190479.3307  1615049976043569
32  6139.7592  190480.4617  1615049976170759
34  6139.7592  190481.5767  1615049976297957
36  6139.7592  190482.6714  1615049976425153
38  6139.7592  190483.8419  1615049976552350
40  6139.7592  190484.9958  1615049976679541
42  6139